In [7]:
import os
import numpy as np
import pandas as pd
import zipfile
import timeit
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.models import Sequential

In [17]:
full_data = pd.read_csv(r"C:\Users\aakas\Downloads\datasets all\train.csv")
test_data_x = pd.read_csv(r"C:\Users\aakas\Downloads\datasets all\test.csv")
test_data_y = pd.read_csv(r"C:\Users\aakas\Downloads\datasets all\test_labels.csv")

In [22]:
full_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [23]:
test_data_x.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [24]:
test_data_y.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [25]:
# Merging the X and y part together
test_dataframe = pd.merge(test_data_X, test_data_y, how="inner", on="id")

# Remove all the rows having missing values (-1)
test_dataframe = test_dataframe[test_dataframe["toxic"] != -1].reset_index(drop=True)

test_dataframe.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
51703,ce2332212289183c,:Wikipedia does not exist to help the souls of...,0,0,0,0,0,0
30875,7b27503319c3ab34,""" \n\n ==So let me understand this== \n *No re...",0,0,0,0,0,0
34945,8b8f25a82c572570,==This type of censorship makes me sick== \n V...,0,0,0,0,0,0
37015,93d984f45fc9498a,""" \n :Agree. The current photo has an """"unveri...",0,0,0,0,0,0
52210,d04eca55b04c3c55,"""Um I didn't say Northern Ireland was not a pa...",0,0,0,0,0,0


In [26]:
print(f"We have {len(test_dataframe)} observations in test data.")

We have 63978 observations in test data.


In [27]:
# Remove the unnecessary data

del test_data_X
del test_data_y

In [28]:
full_data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [29]:
full_data["comment_text"].values

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [30]:
full_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [31]:
# Convert the dataframes into tensorflow Dataset objects

train_data = tf.data.Dataset.from_tensor_slices(
    (
        full_data["comment_text"].tolist(), 
        full_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()
    )
)

test_data = tf.data.Dataset.from_tensor_slices(
    (
        test_dataframe["comment_text"].tolist(), 
        test_dataframe[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()
    )
)

In [32]:
batch_size = 16

train_data = train_data.cache().shuffle(1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_data = test_data.cache().batch(batch_size)

In [33]:
max_tokens = 100000

start = timeit.default_timer()

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    standardize='lower_and_strip_punctuation',
    output_mode="int",
    output_sequence_length=1800
)

text_vectorization.adapt(full_data["comment_text"].tolist())

end = timeit.default_timer()

print(f"It took {end - start} seconds to adapt.")



It took 27.560040200000003 seconds to adapt.


In [34]:
del full_data
del test_dataframe

In [35]:
model = Sequential([
    text_vectorization,
    layers.Embedding(max_tokens+1, 32),
    layers.Bidirectional(layers.LSTM(32, return_sequences=False)),
    layers.Dense(256, activation="relu"),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(6, activation="sigmoid")
])

In [36]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=metrics.BinaryAccuracy()
)

In [37]:
train_split = 0.8

train_data = train_data.take(int(len(train_data) * train_split))
valid_data = train_data.skip(int(len(train_data) * train_split))

In [38]:
print("The cardinality of train data is ", train_data.cardinality().numpy())
print("The cardinality of validation data is ", valid_data.cardinality().numpy())
print("The cardinality of test data is ", test_data.cardinality().numpy())

The cardinality of train data is  7979
The cardinality of validation data is  1596
The cardinality of test data is  3999


In [39]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5, 
        restore_best_weights=True, 
        start_from_epoch=1, 
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", 
        factor=0.5, 
        patience=3, 
        verbose=1, 
        min_lr=1e-6
    )
]

In [ ]:
start = timeit.default_timer()

model_history = model.fit(
    train_data,
    epochs=15,
    verbose=1,
    callbacks=callbacks,
    validation_data=valid_data
).history

end = timeit.default_timer()

print(f"It took {end - start} seconds to train the model.")

Epoch 1/15
7979/7979 [==============================] - 6335s 793ms/step - loss: 0.0613 - binary_accuracy: 0.9793 - val_loss: 0.0452 - val_binary_accuracy: 0.9827 - lr: 0.0010
Epoch 2/15
 727/7979 [=>............................] - ETA: 3:34:15 - loss: 0.0481 - binary_accuracy: 0.9816